# Unidad 07

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [ ]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

## Tablas Cronológicas

In [ ]:
CREATE TABLE Data.Empleados
(
empid INT NOT NULL
CONSTRAINT PK_Empleados PRIMARY KEY NONCLUSTERED,
nombreemp VARCHAR(25) NOT NULL,
departamento VARCHAR(50) NOT NULL,
sueldo NUMERIC(10, 2) NOT NULL,
sysstart DATETIME2(0)
GENERATED ALWAYS AS ROW START HIDDEN NOT NULL,
sysend DATETIME2(0)
GENERATED ALWAYS AS ROW END HIDDEN NOT NULL,
PERIOD FOR SYSTEM_TIME (sysstart, sysend),
INDEX ix_Empleados CLUSTERED(empid, sysstart, sysend)
) WITH (
SYSTEM_VERSIONING =
ON (
HISTORY_TABLE =
Data.EmpleadosHistoria )
);

In [ ]:
ALTER TABLE Data.Empleados ADD
sysstart DATETIME2(0) GENERATED ALWAYS AS ROW START HIDDEN NOT NULL
CONSTRAINT DFT_Empleados_sysstart DEFAULT('19000101'),
sysend DATETIME2(0) GENERATED ALWAYS AS ROW END HIDDEN NOT NULL
CONSTRAINT DFT_Empleados_sysend DEFAULT('99991231 23:59:59'),
PERIOD FOR SYSTEM_TIME (sysstart, sysend);

ALTER TABLE Data.Empleados
SET ( SYSTEM_VERSIONING = ON ( HISTORY_TABLE = Data.EmpleadosHistoria ) );


In [ ]:
SELECT *
FROM Data.Empleados;

SELECT empid, nombreemp, departamento, sueldo, sysstart, sysend
FROM Data.Empleados;


In [ ]:
ALTER TABLE Data.Empleados
ADD fechaalta DATE NOT NULL
CONSTRAINT DFT_Empleados_fechaalta DEFAULT('19000101');

ALTER TABLE Data.Empleados
DROP CONSTRAINT DFT_Empleados_fechaalta;

ALTER TABLE Data.Empleados
DROP COLUMN fechaalta;

In [ ]:
INSERT INTO Data.Empleados(empid, nombreemp, departamento, sueldo)
VALUES
(1, 'Sara', 'IT' , 50000.00),
(2, 'Don' , 'HR' , 45000.00),
(3, 'Judy', 'Sales' , 55000.00),
(4, 'Yael', 'Marketing', 55000.00),
(5, 'Sven', 'IT' , 45000.00),
(6, 'Paul', 'Sales' , 40000.00);

In [ ]:
SELECT empid, nombreemp, departamento, sueldo, sysstart, sysend
FROM Data.Empleados;

SELECT empid, nombreemp, departamento, sueldo, sysstart, sysend
FROM Data.EmpleadosHistoria;


In [ ]:
DELETE FROM Data.Empleados
WHERE empid = 6;

In [ ]:
UPDATE Data.Empleados
SET sueldo *= 1.05
WHERE departamento = 'IT';

In [ ]:
BEGIN TRAN;
UPDATE Data.Empleados
SET departamento = 'Sales'
WHERE empid = 5;

WAITFOR DELAY '00:00:15';

UPDATE Data.Empleados
SET departamento = 'IT'
WHERE empid = 3;
COMMIT TRAN;

In [ ]:
SELECT *
FROM Data.Empleados;

In [ ]:
-- Eliminamos las tablas si existen
IF OBJECT_ID(N'Data.Empleados', N'U') IS NOT NULL
BEGIN
IF OBJECTPROPERTY(OBJECT_ID(N'Data.Empleados', N'U'), N'TableTemporalType') = 2
ALTER TABLE Data.Empleados SET ( SYSTEM_VERSIONING = OFF );
DROP TABLE IF EXISTS Data.EmpleadosHistoria, Data.Empleados;
END;

GO
-- Creamos y populamos las tablas
CREATE TABLE Data.Empleados
(
empid INT NOT NULL
CONSTRAINT PK_Empleados PRIMARY KEY NONCLUSTERED,
nombreemp VARCHAR(25) NOT NULL,
departamento VARCHAR(50) NOT NULL,
sueldo NUMERIC(10, 2) NOT NULL,
sysstart DATETIME2(0) NOT NULL,
sysend DATETIME2(0) NOT NULL,
INDEX ix_Empleados CLUSTERED(empid, sysstart, sysend)
);
GO
INSERT INTO Data.Empleados(empid, nombreemp, departamento, sueldo, sysstart, sysend)
VALUES
(1 , 'Sara', 'IT' , 52500.00, '2016-02-16 17:20:02', '9999-12-31 23:59:59'),
(2 , 'Don' , 'HR' , 45000.00, '2016-02-16 17:08:41', '9999-12-31 23:59:59'),
(3 , 'Judy', 'IT' , 55000.00, '2016-02-16 17:28:10', '9999-12-31 23:59:59'),
(4 , 'Yael', 'Marketing', 55000.00, '2016-02-16 17:08:41', '9999-12-31 23:59:59'),
(5 , 'Sven', 'Sales' , 47250.00, '2016-02-16 17:28:10', '9999-12-31 23:59:59');

-- Creamos y populamos la tabla EmpleadosHistoria
CREATE TABLE Data.EmpleadosHistoria
(
empid INT NOT NULL,
nombreemp VARCHAR(25) NOT NULL,
departamento VARCHAR(50) NOT NULL,
sueldo NUMERIC(10, 2) NOT NULL,
sysstart DATETIME2(0) NOT NULL,
sysend DATETIME2(0) NOT NULL,
INDEX ix_EmpleadosHistoria CLUSTERED(sysend, sysstart)
WITH (DATA_COMPRESSION = PAGE)
);

INSERT INTO Data.EmpleadosHistoria(empid, nombreemp, departamento, sueldo, sysstart, sysend) VALUES
(6 , 'Paul', 'Sales' , 40000.00, '2016-02-16 17:08:41', '2016-02-16 17:15:26'),
(1 , 'Sara', 'IT' , 50000.00, '2016-02-16 17:08:41', '2016-02-16 17:20:02'),
(5 , 'Sven', 'IT' , 45000.00, '2016-02-16 17:08:41', '2016-02-16 17:20:02'),
(3 , 'Judy', 'Sales' , 55000.00, '2016-02-16 17:08:41', '2016-02-16 17:28:10'),
(5 , 'Sven', 'IT' , 47250.00, '2016-02-16 17:20:02', '2016-02-16 17:28:10');

-- Habilitamos el versionado
ALTER TABLE Data.Empleados ADD PERIOD FOR SYSTEM_TIME (sysstart, sysend);
ALTER TABLE Data.Empleados ALTER COLUMN sysstart ADD HIDDEN;
ALTER TABLE Data.Empleados ALTER COLUMN sysend ADD HIDDEN;
ALTER TABLE Data.Empleados
SET ( SYSTEM_VERSIONING = ON ( HISTORY_TABLE = Data.EmpleadosHistoria ) );

In [ ]:
SELECT *
FROM Data.Empleados FOR SYSTEM_TIME AS OF '2016-02-16 17:00:00';

In [ ]:
SELECT *
FROM Data.Empleados FOR SYSTEM_TIME AS OF '2016-02-16 17:10:00';

In [ ]:
SELECT T2.empid, T2.nombreemp,
CAST( (T2.sueldo / T1.sueldo - 1.0) * 100.0 AS NUMERIC(10, 2) ) AS pct
FROM Data.Empleados FOR SYSTEM_TIME AS OF '2016-02-16 17:10:00' AS T1
INNER JOIN Data.Empleados FOR SYSTEM_TIME AS OF '2016-02-16 17:25:00' AS T2
ON T1.empid = T2.empid
AND T2.sueldo > T1.sueldo;

In [ ]:
SELECT empid, nombreemp, departamento, sueldo, sysstart, sysend
FROM Data.Empleados
FOR SYSTEM_TIME FROM '2016-02-16 17:15:26' TO '2016-02-16 17:20:02';

In [ ]:
SELECT empid, nombreemp, departamento, sueldo, sysstart, sysend
FROM Data.Empleados
FOR SYSTEM_TIME BETWEEN '2016-02-16 17:15:26' AND '2016-02-16 17:20:02';

In [ ]:
SELECT empid, nombreemp, departamento, sueldo, sysstart, sysend
FROM Data.Empleados
FOR SYSTEM_TIME CONTAINED IN('2016-02-16 17:00:00', '2016-02-16 18:00:00');

In [ ]:
SELECT empid, nombreemp, departamento, sueldo, sysstart, sysend
FROM Data.Empleados FOR SYSTEM_TIME ALL;

In [ ]:
SELECT empid, nombreemp, departamento, sueldo,
sysstart AT TIME ZONE 'UTC' AT TIME ZONE 'Argentina Standard Time' AS sysstart,
CASE
WHEN sysend = '9999-12-31 23:59:59'
THEN sysend AT TIME ZONE 'UTC'
ELSE sysend AT TIME ZONE 'Argentina Standard Time'
END AS sysend
FROM Data.Empleados FOR SYSTEM_TIME ALL;